# Analyse descriptive des codes MeSH
Ce notebook calcule :
* la similarité (Jaccard) entre `pubmed_mesh` et `mesh_from_gliner` pour chaque cas clinique ;
* des statistiques descriptives (min, max, moyenne, écart‑type, quartiles) sur le **nombre** de codes dans chaque colonne.

> **Dataset utilisé :** `clairedhx/edu3-clinical-fr-mesh-2` (split `train`). Assurez‑vous d’avoir installé `datasets` : `pip install datasets`.

In [1]:
from datasets import load_dataset
import pandas as pd, numpy as np


/home/cldechau/Documents/AOC/codes/AOC_0/aoc-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("clairedhx/edu3-clinical-fr-mesh-2", split="train")
print("Nombre d'exemples :", len(ds))


Generating train split: 100%|██████████| 309/309 [00:00<00:00, 24883.65 examples/s]

Nombre d'exemples : 309


In [3]:
def jaccard(a, b):
    if not a and not b:
        return np.nan
    inter = len(set(a) & set(b))
    union = len(set(a) | set(b))
    return inter / union if union else np.nan

jaccards = []
n_gliner = []
n_pubmed = []

for ex in ds:
    m_g = ex["mesh_from_gliner"]
    m_p = ex["pubmed_mesh"]
    n_gliner.append(len(m_g))
    n_pubmed.append(len(m_p))
    jaccards.append(jaccard(m_g, m_p))

df = pd.DataFrame({
    "jaccard": jaccards,
    "n_gliner": n_gliner,
    "n_pubmed": n_pubmed
})
df.head()


,jaccard,n_gliner,n_pubmed
0,0.166667,8,6
1,0.142857,5,11
2,0.058824,10,8
3,0.125000,2,7
4,0.000000,5,0


In [4]:
print("### Statistiques descriptives Jaccard")
display(df["jaccard"].describe())

print("\n### Nombre de codes MeSH (GLiNER)")
display(df["n_gliner"].describe())

print("\n### Nombre de codes MeSH (PubMed)")
display(df["n_pubmed"].describe())


### Statistiques descriptives Jaccard


count    309.000000
mean       0.078070
std        0.077960
min        0.000000
25%        0.000000
50%        0.071429
75%        0.125000
max        0.428571
Name: jaccard, dtype: float64


### Nombre de codes MeSH (GLiNER)


count    309.000000
mean       5.818770
std        2.986904
min        1.000000
25%        4.000000
50%        5.000000
75%        8.000000
max       18.000000
Name: n_gliner, dtype: float64


### Nombre de codes MeSH (PubMed)


count    309.000000
mean       6.556634
std        3.350838
min        0.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       14.000000
Name: n_pubmed, dtype: float64

In [8]:
# nombre de lignes avec 0 codes MESH recup de Pubmed
empty_pubmed = ds.filter(lambda ex: len(ex["pubmed_mesh"]) == 0)

print(f"{len(empty_pubmed)} lignes sur {len(ds)} n'ont aucun code MeSH PubMed")

Filter: 100%|██████████| 309/309 [00:00<00:00, 16441.78 examples/s]

47 lignes sur 309 n'ont aucun code MeSH PubMed
